# This sample was inspired by [00-getting-started.ipynb](https://github.com/microsoft/semantic-kernel/blob/main/python/notebooks/00-getting-started.ipynb)

## [How does Python SK compare to the C# version of Semantic Kernel?](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md)
- The two SDKs are compatible and at the core they follow the same design principles.
- Some features are still available only in the C# version, and being ported.
- Refer to the [FEATURE MATRIX](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages) doc to see where things stand in matching the features and functionality of the main SK branch.
- Over time there will be some features available only in the Python version, and others only in the C# version, for example adapters to external services, scientific libraries, etc.
<br/>
- Documentation
 - [Get Started with Semantic Kernel](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md)

Inspired to
 - https://github.com/microsoft/semantic-kernel/blob/main/dotnet/samples/KernelSyntaxExamples/Example59_OpenAIFunctionCalling.cs
 - https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/openai_function_calling.py

In [1]:
# create the kernel

import semantic_kernel as sk
kernel = sk.Kernel()

In [2]:
# create the SK "chat completion connector" to the Azure OpenAI service

from dotenv import load_dotenv
load_dotenv("credentials_my.env")

import semantic_kernel.connectors.ai.open_ai as sk_oai

chat_connector = sk_oai.AzureChatCompletion(
    api_key=os.environ['AZURE_OPENAI_API_KEY'],
    api_version=os.environ['AZURE_OPENAI_API_VERSION'],
    deployment_name=os.environ['GPT35TURBO-0613-4k'], # ['GPT4-1106-128k'],
    endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
)
print(f"This is my AzureChatCompletion connector:\n{chat_connector}")

This is my AzureChatCompletion connector:
ai_model_id='gpt35turbo-0613-4k' client=<openai.lib.azure.AsyncAzureOpenAI object at 0x7fbd39b5baa0> ai_model_type=<OpenAIModelTypes.CHAT: 'chat'> prompt_tokens=0 completion_tokens=0 total_tokens=0


In [3]:
print(f"Here are all chat services registered with this kernel: {kernel.all_chat_services()}")

Here are all chat services registered with this kernel: []


In [4]:
# add the openAI completion connector to the kernel, choosing a name to identify it within the kernel

kernel.add_chat_service(
    "mauromi_chatgpt", # unique name to be registered with the kernel
    chat_connector
)
print(f"Here are all chat services registered with this kernel: {kernel.all_chat_services()}")

Here are all chat services registered with this kernel: ['mauromi_chatgpt']


# Load a plugin into the kernel and extract the function from the plugin
## The joke plugin in the FunPlugins is a semantic plugin and has the function calling disabled.

### config.json
```
{
  "schema": 1,
  "description": "Generate a funny joke",
  "execution_settings": {
    "default": {
      "max_tokens": 1000,
      "temperature": 0.9,
      "top_p": 0.0,
      "presence_penalty": 0.0,
      "frequency_penalty": 0.0
    }
  },
  "input_variables": [
    {
      "name": "input",
      "description": "Joke subject",
      "default": ""
    },
    {
      "name": "style",
      "description": "Give a hint about the desired joke style",
      "default": ""
    }
  ]
}
```

### skprompt.txt
```
WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE TOPIC BELOW

JOKE MUST BE:
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY

BE CREATIVE AND FUNNY. I WANT TO LAUGH.
Incorporate the style suggestion, if provided: {{$style}}
+++++

{{$input}}
+++++
```

In [5]:
# load a plugin into the kernel and extract the function from the plugin
# the joke plugin in the FunPlugins is a semantic plugin and has the function calling disabled.

import os
plugins_directory = os.path.join(os.getcwd(), "./Plugins")
plugin = kernel.import_semantic_plugin_from_directory(plugins_directory, "FunPlugin")
joke_function = plugin["Joke"]

print(f"Here are my plugins imported into the kernel:\n{kernel.plugins}")

Here are my plugins imported into the kernel:
data={'funplugin': {'joke': KernelFunction()}}


In [6]:
%%time

# Option 1: run the plugin as a semantic function using context_variables

context_variables = sk.ContextVariables()
context_variables["style"] = "Tell the story in Italian"
result1 = joke_function(input="time travel to dinosaur age", variables=context_variables)
print (result1)

Un giorno, un uomo italiano di nome Giovanni decise di fare un viaggio nel tempo fino all'era dei dinosauri. Era così eccitato all'idea di vedere questi enormi rettili preistorici con i suoi stessi occhi!

Dopo aver attivato la sua macchina del tempo, Giovanni si ritrovò improvvisamente circondato da una giungla lussureggiante e da un caldo soffocante. Mentre camminava con cautela, sentì un rumore provenire da dietro un albero. Si voltò e vide un enorme dinosauro T-Rex che si avvicinava a lui!

Giovanni, spaventato, iniziò a correre per la sua vita. Ma il T-Rex era veloce e sembrava sempre più vicino. Improvvisamente, Giovanni inciampò su una radice e cadde a terra. Si aspettava il peggio, ma quando aprì gli occhi, vide il T-Rex che si era fermato proprio davanti a lui.

Il dinosauro sembrava confuso e un po' triste. Giovanni, ancora tremante, si alzò e disse: "Scusa, signor T-Rex, non volevo disturbarti. Sono solo un turista del tempo che cercava di ammirare la tua bellezza preistoric

In [7]:
%%time

# Option 2: run the plugin as a semantic function using context

context = kernel.create_new_context()
context.variables["style"] = "Tell the story in Italian"
result2 = joke_function(input="time travel to dinosaur age", context=context)
print (result2)

Un giorno, un uomo italiano di nome Giovanni decise di fare un viaggio nel tempo fino all'era dei dinosauri. Era così eccitato all'idea di vedere questi enormi rettili preistorici con i suoi stessi occhi!

Dopo aver attivato la sua macchina del tempo, Giovanni si ritrovò improvvisamente circondato da una giungla lussureggiante e da un caldo soffocante. Mentre camminava con cautela, sentì un rumore provenire da dietro un albero. Si voltò e vide un enorme dinosauro T-Rex che si avvicinava a lui!

Giovanni, spaventato, iniziò a correre per la sua vita. Ma il T-Rex era veloce e sembrava sempre più vicino. Improvvisamente, Giovanni inciampò su una radice e cadde a terra. Si aspettava il peggio, ma quando aprì gli occhi, vide il T-Rex che si era fermato proprio davanti a lui.

Il dinosauro sembrava confuso e un po' triste. Giovanni, ancora tremante, si alzò e disse: "Scusa, signor T-Rex, non volevo disturbarti. Sono solo un turista del tempo che cercava di ammirare la tua bellezza preistoric